Gerekli Kütüphaneler

Aşağıdaki kütüphaneler proje için gereklidir:

openai → OpenAI API kullanımı için

ollama → Lokal Ollama modelleri için

gradio → Web arayüzü oluşturmak için

python-dotenv → API anahtarlarını .env dosyası ile yönetmek için

requests → Web sayfası içeriklerini çekmek için

beautifulsoup4 → HTML içeriğini parse etmek için

In [11]:
pip install openai ollama gradio python-dotenv requests beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


📦 Gerekli Kütüphanelerin İçe Aktarılması

Bu bölümde proje boyunca kullanılacak temel kütüphaneler yüklenir.

Ortam değişkenleri (dotenv)

OpenAI ve Ollama modelleri

Web sitesi içeriklerini çeken scraper

Arayüz için Gradio

In [4]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from scraper import fetch_website_contents
import ollama
import gradio as gr


🧠 Sistem Mesajı (Prompt Engineering)

Bu sistem mesajı, dil modellerine nasıl bir rol üstleneceklerini ve
nasıl bir çıktı üretmeleri gerektiğini tanımlar.

Modelden istenen:

Web sitesini analiz etmesi

Şirket için kısa, pazarlama odaklı bir broşür üretmesi

Markdown formatında yanıt vermesi

Bu yaklaşım, deneysel (experimental) prompt tuning örneğidir.

In [5]:

system_message = """
Bir şirketin web sitesi açılış sayfasının içeriğini analiz eden
ve potansiyel müşteriler, yatırımcılar ve işe alım adayları için
şirket hakkında kısa bir broşür oluşturan bir asistansın.
Kod blokları kullanmadan, Markdown formatında yanıt ver.
"""

🤖 GPT Modeli ile Streaming Yanıt Alma

Bu fonksiyon:

OpenAI GPT modelini kullanır

Yanıtı stream (parça parça) alır

Gradio arayüzünde gerçek zamanlı yazı hissi verir

yield kullanımı sayesinde kullanıcı yanıtı beklemeden çıktıyı görür.

In [6]:
def stream_gpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4.1-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result


🦙 Ollama (Local LLM) ile Streaming Yanıt Alma

Bu fonksiyon:

Yerel çalışan Ollama modelini kullanır

İnternetsiz / offline senaryolar için uygundur

Streaming mantığı GPT ile aynıdır

In [7]:
def stream_ollama(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = ollama.chat(
        model='llama3.2:latest',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk["message"]["content"]
        yield result

🧾 Şirket Broşürü Oluşturma Mantığı

Bu ana fonksiyon:

Şirket adı ve URL alır

Web sitesinin açılış sayfasını scraper ile çeker

Seçilen modele göre (GPT / Ollama) broşür üretir

Prompt otomatik oluşturulur, kullanıcıdan ekstra metin istenmez.

In [8]:
def stream_brochure(company_name, url, model):
    yield ""
    prompt = f"{company_name} için bir şirket broşürü oluşturur musun? İşte açılış (landing) sayfaları:\n"
    #prompt u hazır verdik otomatik oluşturuyor ekstra yazmaya gerek yok

    prompt += fetch_website_contents(url)
    if model=="GPT":
        result = stream_gpt(prompt)
    elif model=="Ollama":
        result = stream_ollama(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

🖥️ Gradio Arayüz Bileşenleri

Bu bölümde kullanıcıdan alınacak girişler tanımlanır:

Şirket adı

Web sitesi URL’si

Kullanılacak model seçimi

Çıktı Markdown formatında gösterilir.

In [9]:
name_input = gr.Textbox(label="Şirket adı:")
url_input = gr.Textbox(label="http:// veya https:// dahil açılış sayfası URL’si")
model_selector = gr.Dropdown(["GPT", "Ollama"], label="Model seçin", value="GPT")
message_output = gr.Markdown(label="Yanıt:")

🚀 Gradio Arayüzünün Oluşturulması ve Çalıştırılması

Bu arayüz:

Örnek şirketler içerir

Streaming yanıtları destekler

Basit bir kullanıcı adı / şifre ile korunur

Tarayıcıda otomatik açılır

In [13]:
view = gr.Interface(
    fn=stream_brochure,
    title="Broşür Oluşturucu",
    inputs=[name_input, url_input, model_selector],
    outputs=[message_output],
    examples=[
        ["Hugging Face", "https://huggingface.co", "GPT"],
        ["OpenAI", "https://openai.com", "Ollama"]
    ],
    flagging_mode="never"
)

#auth parametresi launch() içinde verilir.
view.launch( 
    inbrowser=True,
    auth=("muzo", "123qwe")
)


* Running on local URL:  http://127.0.0.1:7884
* To create a public link, set `share=True` in `launch()`.
